In [1]:
from pycaret.datasets import get_data

import pickle
def load_pkl(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
from pycaret.classification import *
import tqdm
import pandas as pd
name= '3'

juice = pd.read_csv(f'./data{name}.csv')
exp = ClassificationExperiment()

train_df = juice.head(100)
test_df = juice.tail(60)

In [3]:
import pycaret

In [4]:
pycaret.__version__

'3.0.4'

In [144]:
juice.head(100).label.value_counts()

0.0    77
1.0    23
Name: label, dtype: int64

In [147]:
juice

,饮酒史,HM_NCCT_original_firstorder_Maximum,冠心病史,ED_NCCT_original_firstorder_InterquartileRange,HM_NCCT_original_firstorder_Skewness,止吐护胃,HM_original_shape_Elongation,ED_PCA_R_Ratio,止血治疗,ED_original_shape_Elongation,label
0,0.0,0.495179,0.0,0.623458,0.073091,1.0,0.707550,0.138865,1.0,0.690562,0.0
1,0.0,0.365019,0.0,0.384013,0.432276,1.0,0.605696,0.251132,1.0,0.890860,0.0
2,0.0,0.387395,0.0,0.627288,0.379049,1.0,0.492735,0.076306,1.0,0.863232,1.0
3,0.0,0.716242,0.0,0.781738,0.550780,0.0,0.215563,0.012698,1.0,0.300077,0.0
4,0.0,0.330957,0.0,0.844969,0.383662,1.0,0.933752,0.011891,1.0,0.515068,1.0
...,...,...,...,...,...,...,...,...,...,...,...
155,0.0,0.473792,1.0,0.990488,0.164114,1.0,0.469155,0.000000,1.0,0.870203,NaN
156,0.0,0.346794,0.0,0.119625,0.577202,1.0,0.520439,0.000000,0.0,0.415720,NaN
157,0.0,0.342428,0.0,0.518757,0.436945,1.0,0.704972,0.163457,1.0,0.845392,NaN
158,1.0,0.288904,0.0,0.714367,0.380374,1.0,0.889654,0.000000,1.0,0.502884,NaN


In [141]:
exp.models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [ ]:
len(list(exp.models().reset_index()['ID']))
top5list = {}
for random_state in tqdm.tqdm(range(34,45,1)):
    exp.set_config('seed', random_state)
    exp_name = exp.setup(data = train_df,  target = 'label',verbose=False)
    top5 = exp.compare_models(sort = 'auc',n_select = 5,budget_time=360,verbose=True,include=list(exp.models().reset_index()['ID']))
    top5list[random_state] = [top5,exp._display_container[1] ] 

In [5]:
len(top5list)

44

In [42]:
def save_pkl(path,obj):
    with open(path, 'wb') as f:
        pickle.dump(obj,f)
        
def load_pkl(path): 
    with open(path, 'rb') as f:
        return pickle.load(f) 
#save_pkl(f'{name}_2.pkl',top5list)

In [117]:
all_result = pd.DataFrame()
for i in top5list.keys():
    t = top5list[i][1]
    t['random'] = i
    all_result=all_result.append(t)

In [118]:
all_result.Model.unique(),all_result.shape[0]

(array(['Logistic Regression', 'MLP Classifier', 'Extra Trees Classifier',
        'Naive Bayes', 'SVM - Radial Kernel',
        'Linear Discriminant Analysis', 'Gaussian Process Classifier',
        'CatBoost Classifier', 'Ada Boost Classifier',
        'Random Forest Classifier', 'K Neighbors Classifier',
        'Decision Tree Classifier', 'Light Gradient Boosting Machine',
        'Gradient Boosting Classifier', 'Dummy Classifier',
        'Extreme Gradient Boosting', 'SVM - Linear Kernel',
        'Ridge Classifier', 'Quadratic Discriminant Analysis'],
       dtype=object),
 782)

In [119]:
len(all_result.Model.unique())

19

In [120]:
# all_result = all_result[all_result.Accuracy>.5]
# all_result = all_result[all_result.AUC>.5]
# all_result.shape[0]

In [121]:
top5list[0][0]

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=4614, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
               beta_2=0.999, early_stopping=False, epsilon=1e-08,
               hidden_layer_sizes=(100,), learning_rate='constant',
               learning_rate_init=0.001, max_fun=15000, max_iter=500,
               momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
               power_t=0.5, random_state=4614, shuffle=True, solver='adam',
               tol=0.0001, validation_fraction=0.1, verbose=False,
               warm_start=False),
 ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                      criterion='gini', max_depth=None, max

In [122]:
## 所有表现的
cols = ['AUC','Accuracy','Recall','Prec.']
grouped = all_result.groupby('Model')
def mean(values):
    mean = values.mean()
    return mean
result2 = grouped.apply(lambda x: x.nlargest(3, columns=cols)).reset_index(drop=True)

In [124]:
result1 = all_result[all_result.random.isin(result2[result2.Model == 'MLP Classifier'].random)]
result1 =result1.groupby('Model').agg(mean)
result1.sort_values(cols[0],ascending=False)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec),random
Model,,,,,,,,,
MLP Classifier,0.852400,0.907767,0.633333,0.688900,0.621100,0.548900,0.576233,0.480667,30.0
SVM - Radial Kernel,0.771400,0.855533,0.000000,0.000000,0.000000,0.000000,0.000000,0.443333,30.0
Naive Bayes,0.376167,0.850000,0.983333,0.274600,0.416033,0.105233,0.196900,0.450667,30.0
Linear Discriminant Analysis,0.814267,0.841100,0.533333,0.605533,0.531100,0.434500,0.459767,0.447333,30.0
Extra Trees Classifier,0.838133,0.836667,0.550000,0.608333,0.548900,0.475100,0.493867,0.509333,30.0
Gaussian Process Classifier,0.776167,0.836667,0.016667,0.033333,0.022233,0.019600,0.021500,0.543333,30.0
K Neighbors Classifier,0.809533,0.834467,0.516667,0.552767,0.486667,0.403000,0.433667,0.452667,30.0
Logistic Regression,0.771400,0.833333,0.016667,0.033333,0.022233,0.014067,0.015967,0.461667,30.0
Gradient Boosting Classifier,0.785700,0.821100,0.533333,0.541667,0.496667,0.390600,0.415533,0.455333,30.0


In [139]:
## 所有表现的
cols = ['AUC','Accuracy','Recall','Prec.']
grouped = all_result.groupby('Model')
def mean(values):
    mean = values.mean()
    std = values.std()
    return f"{mean:.2f} ± {std:.2f}"
result2 = grouped.apply(lambda x: x.nlargest(3, columns=cols)).reset_index(drop=True)

result1 = all_result[all_result.random.isin(result2[result2.Model == 'MLP Classifier'].random)]
result1 =result1.groupby('Model').agg(mean)
result1.sort_values(cols[0],ascending=False).to_csv('./model_result.csv',index=True,encoding='utf-8-sig')

In [135]:
all_result[ all_result.Model == 'MLP Classifier' ].sort_values(cols[0],ascending=False)  #选择24

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec),random
mlp,MLP Classifier,0.8429,0.9333,0.60,0.6500,0.5833,0.5075,0.5361,0.464,23
mlp,MLP Classifier,0.8429,0.9000,0.65,0.7000,0.6333,0.5579,0.5872,0.449,24
mlp,MLP Classifier,0.8000,0.8900,0.45,0.4833,0.4400,0.3285,0.3461,0.446,20
mlp,MLP Classifier,0.8714,0.8900,0.65,0.7167,0.6467,0.5813,0.6054,0.529,43
mlp,MLP Classifier,0.7857,0.8667,0.45,0.4333,0.4167,0.3150,0.3292,0.450,22
mlp,MLP Classifier,0.8143,0.8600,0.45,0.4167,0.3967,0.3349,0.3612,0.509,40
mlp,MLP Classifier,0.8000,0.8600,0.40,0.4333,0.4067,0.3040,0.3079,0.443,18
mlp,MLP Classifier,0.8000,0.8533,0.60,0.5083,0.4900,0.4351,0.4761,0.458,25
mlp,MLP Classifier,0.7429,0.8533,0.30,0.2833,0.2900,0.1818,0.1800,0.480,29
mlp,MLP Classifier,0.8143,0.8500,0.50,0.5833,0.5067,0.4038,0.4248,0.410,4


In [137]:
result = exp.predict_model(top5list[24][0][0],data=juice.iloc[:,:-1],raw_score=True)['prediction_score_1'].round(4)
result.to_csv('result1b.csv')